In [2]:
from typing import List

import os
os.environ['https_proxy'] = "10.192.132.179:10811"


import json
from openai import OpenAI

config = json.load(open("./secret_config.json"))


client = OpenAI(api_key=config["openai_key"], base_url="https://api.132999.xyz/v1/")

def get_gpt_response(user_texts: List[str]):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
            {
                "role": "system", 
                "content": f"你是一个智慧的商业分析员，你会根据用户购买商品后的评价来判断其是否是消极或者积极；并且选择出用户评价的主要因素，\
                    从（造型、颜色、图案、风格、尺寸、包装、材质、工艺、实用性、文化、情感、审美、品质、物流、价格）这里面选一个产生评价的最主要因素（若无法确定，则为其他）。\
                    接下来我将会每一行输入一个用户评价。你将返回你的分析结果，每一行分别返回：[原始评价];[积极或消极];[因素]。\
                    用【英文分号，也就是';'】分隔。例如对于\"太贵了\"这条评价，你将会返回\"太贵了;消极;价格\""
            },
            {
                "role": "user",
                "content": "\n".join(user_texts)
            }
    ])
    try:
        resps = response.choices[0].message.content.split("\n")
        if len(resps) != len(user_texts):
            resps += (len(user_texts) - len(resps)) * [""]
            print("注：此时GPT的回复数缺失！已经靠后补全")
        return resps
    except:
        return []

In [1]:
import pandas as pd


In [ ]:
df = pd.read_csv("data/故宫纪念品评价-清洗后.csv", encoding="utf-8", lineterminator='\n')
df.loc[:, 'content'] = df['content'].astype(str)

In [ ]:
df['gpt_emotion'] = ""
df['gpt_factor'] = ""

In [ ]:
# 填补未产生GPT回复的

df['content'] = df['content'].astype(str)
df['content'] = df['content'].str.strip()
df['content'] = df['content'].str.replace('\x01', '')
df['content'] = df['content'].str.replace('\n', ' ')

In [3]:
df = pd.read_csv("data/ChatGPT分析结果.csv", dtype={
    "content": 'string', 'gpt_emotion': 'string', 'gpt_factor': 'string'
}, keep_default_na=False, index_col=0)
df = df[(df['content'] !=  "此用户没有填写评价。") & (df['content'] != "")]

print("Unprocessed: ", (df['gpt_emotion']=="").sum())
i_start = 0
while True:
    unparsed_indices = []
    unparsed_contents = []
    for i in range(i_start, len(df)):
        if df.loc[i, 'gpt_emotion'] == "":
            unparsed_indices.append(i)
            unparsed_contents.append(df.loc[i, 'content'])
            if len(unparsed_indices) == 10:
                break
    i_start = i + 1

    print("Contents", unparsed_contents)
    try:
        resps = get_gpt_response(unparsed_contents)
    except Exception as e:
        print(f"Error: {e}")
        continue
    print("Resps", resps)
    resp_dict = dict()
    for r in resps:
        try:
            content, emotion, factor = r.split(";")
            resp_dict[content] = (emotion, factor)
        except:
            print("Fail to parse", r)
    
    current_parsed = 0
    for j in unparsed_indices:
        content = df.loc[j, 'content']
        if content in resp_dict:
            emotion, factor = resp_dict[content]
            df.loc[j, 'gpt_emotion'] = emotion
            df.loc[j, 'gpt_factor'] = factor
            current_parsed += 1
    print(f"Parsed {current_parsed} contents.")
    df.to_csv("data/ChatGPT分析结果.csv")

Unprocessed:  18


KeyError: 540

In [11]:
import time
# 最后几条容易复述失败的模型，单条喂给GPT：
while True:
    unparsed_indices = []
    unparsed_contents = []
    for i in range(0, len(df)):
        if df.loc[i, 'gpt_emotion'] == "":
            unparsed_indices.append(i)
            unparsed_contents.append(df.loc[i, 'content'])
            if len(unparsed_indices) == 1:
                break
    time.sleep(0.5)
    print("Contents", unparsed_contents)
    try:
        resps = get_gpt_response(unparsed_contents)
    except:
        continue
    print("Resps", resps)
    for r in resps:
        try:
            content, emotion, factor = r.split(";")
            df.loc[i, 'gpt_emotion'] = emotion
            df.loc[i, 'gpt_factor'] = factor
        except:
            print("Fail to parse", r)
    df.to_csv("data/ChatGPT分析结果.csv")

Contents ['宝贝收到了，质量很不错，包装精致，材质优秀，比想象中好，送礼自用非常合适，下次继续购买宝贝收到了，质量很不错，包装精致，材质优秀，比想象中好，送礼自用非常合适，下次继续购买宝贝收到了，质量很不错，包装精致，材质优秀，比想象中好，送礼自用非常合适，下次继续购买宝贝收到了，质量很不错，包装精致，材质优秀，比想象中好，送礼自用非常合适，下次继续购买宝贝收到了，质量很不错，包装精致，材质优秀，比想象中好，送礼自用非常合适，下次继续购买宝贝收到了，质量很不错，包装精致，材质优秀，比想象中好，送礼自用非常合适，下次继续购买宝贝收到了，质量很不错，包装精致，材质优秀，比想象中好，送礼自用非常合适，下次继续购买宝贝收到了，质量很不错，包装精致，材质优秀，比想象中好，送礼自用非常合适，下次继续购买宝贝收到了，质量很不错，包装精致，材质优秀，比想象中好，送礼自用非常合适，下次继续购买宝贝收到了，质量很不错，包装精致，材质优秀，比想象中好，送礼自用非常合适，下次继续购买']
Resps ['宝贝收到了，质量很不错，包装精致，材质优秀，比想象中好，送礼自用非常合适，下次继续购买;积极;品质']
Contents ['漂亮，有特色，比较贵的防尘口罩，各两盒共40个要73元，双十一的价钱，']
Resps ['漂亮，有特色，比较贵的防尘口罩，各两盒共40个要73元，双十一的价钱;积极;价格']
Contents ['質量非常好,與賣家描述的完全一致, 真的很喜歡,完全超出期望值,發貨速 度非常快,包裝非常仔細、嚴實,物流公司服務態度很好,運送速度很快,很滿意的一次購物質量很好, 希望更多的朋友信賴. 店主態度特好, 我會再次光顧的好賣家 贊，發貨迅速，態度很好，很滿意！很好很好！網上購物這麼激烈，沒想到店家的服務這麼好，商品質量好而價低廉， 很熱情的賣家，下次還來希望下次還有機會合作祝你生意興隆質量非常好，出乎我的意料包裝非常仔細。下次有機會再找你，店家人蠻好的，東東很不錯,淘到心意的寶貝是一件讓人很開心的事，比心??????????????? 質量非常好,與賣家描述的完全一致, 真的很喜歡,完全超出期望值,發貨速 度非常快,包裝非常仔細、嚴實,物流公司服務態度很好,運送速度很快']
Resps ['質量非常好,與賣家描述的完全一致, 真的很喜歡,完全超出期望值,發貨速度

KeyboardInterrupt: 

In [5]:
df.drop(["Unnamed: 0", "Unnamed: 0.1"])
df.to_csv("data/ChatGPT分析结果.csv")

KeyError: "['Unnamed: 0' 'Unnamed: 0.1'] not found in axis"

In [9]:
df.drop(columns=df.columns[:15])

,product_name,content,reply,gpt_emotion,gpt_factor
0,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,非常漂亮,,积极,审美
1,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,很精致，买来做教师节礼物的,,积极,实用性
2,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,很好看，但是送礼物了，没有打开看，希望朋友喜欢。,,中性,其他
3,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,质量很好，买的很合我心意，棒棒哒！✨需要的亲们可以放心购买,,积极,品质
4,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,送给妹** 她很喜欢,,积极,情感
...,...,...,...,...,...
10204,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,好,,积极,其他
10205,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,漂亮,,积极,造型
10206,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,故宫文创yyds！纸质非常好！,,积极,材质
10207,故宫小确幸笔记本日记本礼品本子生日礼物礼品伴手礼,好评,,积极,其他


In [10]:
df.index = list(range(10202))